## Spark ZH
## NEPTUN: PST8RA

In [1]:
from pyspark import SparkConf, SparkContext 
import findspark
findspark.init()
findspark.find()
conf = SparkConf().set("spark.local.dir", "/home/jxn/.sparktmp") # A local dir nélkül nem működik a configom, de azt ki lehet majd törölni
sc = SparkContext(conf=conf)

23/11/09 08:27:38 WARN Utils: Your hostname, jxn-linux resolves to a loopback address: 127.0.1.1; using 10.10.116.13 instead (on interface wlp4s0)
23/11/09 08:27:38 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/09 08:27:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/11/09 08:27:39 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


23/11/09 08:27:54 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import *
from pyspark.sql.functions import *

spark = SparkSession.builder.getOrCreate()

23/11/09 09:22:39 WARN Utils: Your hostname, jxn-linux resolves to a loopback address: 127.0.1.1; using 10.10.116.13 instead (on interface wlp4s0)
23/11/09 09:22:39 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/09 09:22:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### 1. FELADAT

In [51]:
def k_mer(line):
  kmers = []
  i = 0
  while(i+2 < len(line)):
    kmers.append(line[i:i+3])
    i+=1
  return kmers

kmer_dict = sc.textFile('kmerInput.txt')\
  .map(lambda line : k_mer(line))\
  .flatMap(lambda x : x)\
  .filter(lambda kmer : "T" in kmer)\
  .countByValue()

result = sc.parallelize(kmer_dict.values()).filter(lambda value: value > 100).count()
print(result)

defaultdict(<class 'int'>, {'GCT': 136, 'CTT': 78, 'TTT': 126, 'TTC': 103, 'TCA': 111, 'CAT': 95, 'ATT': 114, 'TCT': 95, 'CTG': 176, 'TGA': 151, 'ACT': 70, 'TGC': 166, 'AAT': 124, 'ATA': 71, 'TAT': 80, 'ATG': 132, 'TGT': 75, 'GTC': 71, 'CTC': 77, 'GTG': 112, 'TGG': 151, 'GAT': 142, 'TTA': 95, 'TAA': 80, 'AGT': 72, 'TAG': 37, 'GGT': 113, 'GTT': 125, 'TAC': 67, 'CCT': 62, 'CGT': 106, 'GTA': 59, 'TTG': 121, 'CTA': 33, 'ATC': 126, 'TCC': 71, 'TCG': 95})
18


### 2. FELADAT

In [65]:
weboldalak = sc.textFile('weboldalak.txt')\
  .map(lambda line: (line.split(" ", 1)[0], line.split(" ", 1)[1]))\
  .filter(lambda web: len(web[1]) > 10)\
  .map(lambda web: (web[0], len(list(filter(lambda x: x == "ELTE" or x == "elte", web[1].split(" ")))), len(web[1].split(" "))))\
  .sortBy(lambda web: web[1], False).take(1)

weboldal = weboldalak[0]

print(weboldal)

('elte.hu', 3, 53)


### 3. FELADAT

In [21]:
df_books = spark.read\
  .format("csv")\
  .option('header', True)\
  .option("inferschema", True)\
  .load("books.csv")
  
df_books.createOrReplaceTempView("BOOKS")
  
spark.sql("SELECT AUTHOR, COUNT(TITLE) FROM BOOKS WHERE AUTHOR IS NOT NULL GROUP BY AUTHOR, TITLE SORT BY COUNT(TITLE) DESC").show(3)

+--------------------+------------+
|              AUTHOR|count(TITLE)|
+--------------------+------------+
|       Lewis Carroll|          15|
|Petra Mettke, Kar...|          11|
|    L. M. Montgomery|          11|
+--------------------+------------+
only showing top 3 rows



### 4. FELADAT

In [31]:
df_books = spark.read\
  .format("csv")\
  .option('header', True)\
  .option("inferschema", True)\
  .load("books.csv")
  
df_books.createOrReplaceTempView("BOOKS")

spark.sql("SELECT AUTHOR, COUNT(PUBLISHER) FROM BOOKS WHERE AUTHOR IS NOT NULL GROUP BY AUTHOR, PUBLISHER HAVING COUNT(PUBLISHER) > 35").show(300)

+--------------------+----------------+
|              AUTHOR|count(PUBLISHER)|
+--------------------+----------------+
|Shelley Admont, K...|              89|
|       Carolyn Keene|              50|
|          Adam Blade|              45|
|    Roger Hargreaves|              56|
|       Die Blokehead|              49|
|         R. L. Stine|              78|
|     Kirsteen Robson|              39|
|       Mohammed Umar|              44|
|         Idries Shah|             160|
|Steve Barlow, Ste...|              43|
|          J. R. Ward|              48|
|         Enid Blyton|             127|
|          Tuula Pere|              52|
|        El Blokehead|              89|
|          Tuula Pere|              40|
|       James Manning|             105|
|         Jake Maddox|              51|
|     Peter Hertzberg|              41|
|         Thithiajobs|              62|
|       Daisy Meadows|              56|
|       Penelope Dyan|              79|
|       Grace Goodwin|              55|
